# Introduction
This notebook is part of a machine learning for healthcare exercise, focusing on using the Responsible AI (RAI) package to enhance clinical decision-making. The toolkit will be used to analyze diabetes progression data, with two key objectives:

1. Analyze Errors and Explore Interpretability of Models. Run Interpret-Community's 'explain_model' globally to generate model explanations and Visualize model errors and global explanations with the Error Analysis visualization dashboard

2. Plan real-world action through counterfactual and causal analysis: By leveraging counterfactual example analysis and causal inferencing, we will explore decision-making strategies based on diabetes progression data to understand potential treatment paths and their impacts

3. Assess disease progression predictions: A regression model trained on diabetes progression data will be assessed, examining its performance in predicting disease progression to inform clinical decisions

**The goal is for you to provide non-trivial insights for clinical decision making, leveraging machine learning, paired with responsible AI tools, to improve patients outcomes in the healthcare contexts.**

Based on notebooks from the [Responsible AI toolkit](https://github.com/microsoft/responsible-ai-toolbox/tree/main/notebooks)

In [1]:
import shap
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_breast_cancer
from interpret.ext.blackbox import TabularExplainer
from raiwidgets import ErrorAnalysisDashboard

Load the [breast cancer diagnosis data](https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset) and specify the different types of features. Then, clean it and put it into a DataFrame with named columns.

Data Set Characteristics:

Number of Instances: 569

Number of Attributes: 30 numeric, predictive attributes and the class

Attribute Information:
- radius (mean of distances from center to points on the perimeter)
- texture (standard deviation of gray-scale values)
- perimeter
- area
- smoothness (local variation in radius lengths)
- compactness (perimeter^2 / area - 1.0)
- concavity (severity of concave portions of the contour)
- concave points (number of concave portions of the contour)
- symmetry
- fractal dimension (“coastline approximation” - 1)

The mean, standard error, and “worst” or largest (mean of the three worst/largest values) of these features were computed for each image, resulting in 30 features. For instance, field 0 is Mean Radius, field 10 is Radius SE, field 20 is Worst Radius.

class:
- WDBC-Malignant ("1")
- WDBC-Benign ("0")

![Tumor Image](img\tumor.png)

In [2]:
breast_cancer_data = load_breast_cancer()
classes = breast_cancer_data.target_names.tolist()

In [ ]:
type(breast_cancer_data)

In [4]:
data = sklearn.datasets.load_breast_cancer()
target_feature = 'y'
continuous_features = data.feature_names
data_df = pd.DataFrame(data.data, columns=data.feature_names)

In [ ]:
data_df.head()

### Train a Model
After loading and cleaning the data, split the datapoints into training and test sets. Assemble separate datasets for the full sample and the test data.

In [6]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(
    data_df, data.target, test_size=0.2, random_state=7)

# Creating train_data and test_data for RAIInsights
train_data = X_train.copy()
test_data = X_test.copy()
train_data[target_feature] = y_train
test_data[target_feature] = y_test

Train a LR (Logistic Regressor) classifier on the training data

In [ ]:
# Initialize the Logistic Regression model
model = LogisticRegression(random_state=0, max_iter=10000)
# Train the model
model.fit(X_train, y_train)

Train a RF (Random-Forest) classifier on the training data

In [8]:
# model = RandomForestClassifier(random_state=0)
# model.fit(X_train, y_train)

### Explain predictions on your local machine

In [ ]:
# 1. Using SHAP TabularExplainer
explainer = TabularExplainer(model,
                             X_train,
                             features=X_train.columns,
                             classes=classes)

### Generate global explanations
Explain overall model predictions (global explanation)

In [ ]:
# Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
# X_train can be passed as well, but with more examples explanations will take longer although they may be more accurate
global_explanation = explainer.explain_global(X_test)

### Visualize
Load the interpretability visualization dashboard

In [11]:
from raiwidgets import ExplanationDashboard

In [ ]:
ExplanationDashboard(
    global_explanation, 
    model, 
    dataset=X_test, 
    true_y=y_test)

### Create Model and Data Insights

In [13]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

[Optional] Defining an "identity_feature". An identity feature in Microsoft's Responsible AI Toolkit helps detect and mitigate bias in sensitive attributes like race, gender, and age for fairness. 

Marking a feature as an identity feature can trigger specific analyses within the toolkit, such as:

- Fairness assessment: Evaluating if certain identity groups are disproportionately affected by model decisions
- Bias detection: Highlighting patterns of bias in model performance across identity groups
- Mitigating unfairness: Providing tools to adjust or retrain models to improve fairness with respect to the identity features

In [14]:
from responsibleai.feature_metadata import FeatureMetadata
# Add 's1' as an identity feature
feature_metadata = FeatureMetadata(
    identity_feature_name=None, 
    categorical_features=[], 
    dropped_features=[])

RAIInsights accepts the model, the train dataset, the test dataset, the target feature string and the task type string as its arguments.

In [15]:
rai_insights = RAIInsights(
    model, 
    train_data, 
    test_data, 
    target_feature, 
    'classification',
    feature_metadata=feature_metadata)

Add the components of the toolbox that are focused on **Model assessment and Decision-making.**

- Explanations (.explainer): Clarifies how model features influence predictions using algorithms like SHAP or LIME
- Error Analysis (.error_analysis): Identifies subgroups of data where the model makes the most mistakes, helping with troubleshooting
- Counterfactuals (.counterfactual): Generates "what-if" scenarios, showing how feature changes affect outcomes
- Causal Inference (.causal): Analyzes whether certain features (e.g., bmi, bp) directly cause changes in the outcome
    - For potential "treatments", we'd use the causal inference component to understand the impact of changing certain features on the outcome:
    - mean_radius (mean of distances from the center to the boundary of the nuclei)
    - mean_texture (standard deviation of gray-scale values)
    - mean_perimeter (average perimeter of the cell nuclei)
    - mean_area (mean area of the cell nuclei)
    - mean_smoothness (mean local variation in radius lengths)


In [16]:
# Interpretability
rai_insights.explainer.add()
# Error Analysis
rai_insights.error_analysis.add()
# Counterfactuals: accepts total number of counterfactuals to generate, the range that their label should fall under, 
# and a list of strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=10, desired_class="opposite")
# Causal Inference: determines causation between features
rai_insights.causal.add(
    treatment_features=[
        'mean radius',
        'mean texture',
        'mean perimeter', 
        'mean smoothness',
        'mean area'
        ]  # Example features
    )  

Once all the desired components have been loaded, we compute insights on the test set

In [ ]:
rai_insights.compute()

Visualize and explore the model insights. Use the resulting widget or follow the link to view this in a new tab

In [ ]:
ResponsibleAIDashboard(rai_insights)

In [ ]:
predictions = model.predict(X_test)
ErrorAnalysisDashboard(dataset=X_test, true_y=y_test,
                       features=continuous_features, pred_y=predictions)